# Test audioplot library

This notebook is a tiny showcase of the core components of the library.


In [1]:
import datetime as dt

import numpy as np

import audible_plot as ap

player = ap.AudioPlayer()
generator = ap.SineToneGenerator()

Here we generate some samples and play them with the player object.

The player object is an intermediate object to abstract audio playback and encoding, you can play the audio the way you like.

The sample format is float64, so you would need to convert it to the format you like before playing it.


In [2]:
sample1 = generator.generate_wave(44100, dt.timedelta(milliseconds=250), 250, 300)
sample2 = generator.generate_wave(44100, dt.timedelta(milliseconds=1000), 300, 400)
sample3 = generator.generate_wave(44100, dt.timedelta(seconds=2), 400, 250)
sample4 = generator.generate_wave(44100, dt.timedelta(seconds=1), 250, 250)
sample5 = generator.generate_wave(44100, dt.timedelta(seconds=2), 250, 440)
sample6 = generator.generate_wave(44100, dt.timedelta(seconds=2), 440)
samples = np.vstack((sample1, sample2, sample3, sample4, sample5, sample6))
player.play_raw(samples)

## Concepts:

- **Chart:** A group of related data series. The requirement of the series is to have the same length.
- **Series:** A sequence of float values. Here we use _NumPY_ to generate the example values.
- **Renderer:** A renderer maps values to frequency and then generates an audio sample from them.
- **Windows:** A window of data. This is needed to get the range of values in a given time.

I have to refactor this to be more usable with NumPY, Pandas and other data-related libraries.


In [3]:
chart = ap.AudibleChart(
    max_freq=800,
    min_freq=300,
    series=[
        ap.AudibleSeries(
            np.linspace(0, 10, 15),
            ap.PitchDataRenderer(
                ap.FixedRange(300, 800),
                ap.SineToneGenerator(),
                pan=-0.7,
                volume=0.2,
                enable_transitions=True,
            ),
            "test1",
        ),
        ap.AudibleSeries(
            np.linspace(2, 9, 15),
            ap.PitchDataRenderer(
                ap.FixedRange(300, 800),
                ap.SineToneGenerator(),
                pan=0.7,
                volume=0.2,
                enable_transitions=True,
            ),
            "test2",
        ),
    ],
)

In [4]:
# Get a window of data from the chart
window = chart.window(0, 15)

# Get a particular value of the chart window
# TODO: Implement `__getitem__` to support this with a less verbose way
window["test2"][-1]

np.float64(9.0)

In [5]:
# Play a chart window
window.play("all")

In [7]:
# Play a particular series:
window.play("test1")

In [8]:
# Give a starting point to a series to play
window.play("test2", slice(10, 15))

I didn't found a way to remove microclips between samples...
